# Bots

## Tipos de Bots de Trading

BOTs de DataFeed / Screeners

BOTs de Trading

    A- Bots de Colocación de grandes órdenes, para no alterar precio
    B- Bots de Colocación de órdenes, para obtener el mejor precio
    C- Bots de Market Making
    D- Bots de Arbitrajes en un mercado
    E- Bots de Arbitrajes entre mercados
    F- Bots de Arbitrajes estadísticos (en un mercado o entre mercados)
    G- Bots de Hedging
    H- Bots de Balanceo de carteras (optimización, riesgo, etc)
    I- Bots de ruteo para seguimiento de índices (fondos, etfs)
    J- Bots de scalping
    K- Bots de swing trading


BOTs estratégicos

* Bots de detección de otros bots

# Planteo de nuestro bot de scalping

Caracteristicas:
    
    Tipo operacion: Swing
    Timerframe: 1min
    Instrumento GGAL
    Señal Compra; Modelo ML
    Señal Vta: timeout
    Tipo de compra y venta: Agresivo
    Driver: Mercado USA
    Ruteo: Mercado Arg
     
Acciones:
    
    Lectura de datos USA => API tiingo
    Operatoria en Arg => API IOL
    Prediccion => Modelo de ML
    Decision de Venta => Lectura del reloj y BBDD con registro de compra
    Persistencia => A BBDD


# Funciones Previas

## Importamos todas las lisbrerias y claves

In [2]:
from iol_user_ullua import *

iol_user = USER
iol_password = PASS

In [3]:
import datetime, requests, time
import pandas as pd
import numpy as np
from IPython.display import clear_output
import pickle

In [4]:
from sqlalchemy import create_engine

## Funciones ML y Data USA

In [5]:
def traerModelo(tipo='RF'):
    if tipo=='RF':
        with open('bot_rf.dat', 'rb') as file:
            modelo = pickle.load(file)
    else: 
        modelo = None

    return modelo

In [6]:
def downloadTiingo(symbol, _from, _to, interval='1min'):
    #clear_output(wait=True)
    try:
        TOKEN = '3e63d74be68b49e14fb4c161c378b6250ad8d5c3'
        url = f'https://api.tiingo.com/iex/{symbol}/prices'
        headers = {'Content-Type': 'application/json'}
        params = {'startDate':_from, 'endDate':_to, 'resampleFreq':interval, 'token':TOKEN }
        r = requests.get(url, params=params, headers=headers)
        df = pd.DataFrame(r.json()).set_index('date')
        #print('Datos recabados ok hasta el: ', _to)
        return df
    except:
        print(f'No se pudo descargar desde Tiingo data de {symbol}')
        return None

In [7]:
def generarIndicadores(data):
    try:
        df = (data['close']+data['open']+data['high']+data['low'])/4
        cruces = [(2,20),(2,40),(2,60),(2,100),(2,200),(5,20),(5,50),(5,100),(5,200),(5,400),(10,20),(10,50),(10,100),
                 (10,200),(10,500),(20,50),(20,100),(20,200),(20,500),(20,1000),(50,100),(50,200),(50,500),(50,1000),
                 (100,200),(100,400),(100,500),(100,1000),(200,500),(200,1000),(400,1000)]

        df = df.to_frame().apply(pd.to_numeric)
        df.columns = ['px']
        for cruce in cruces:
            clave = str(cruce[0]) + '_' + str(cruce[1])
            df[clave] = (df.px.rolling(cruce[0]).mean() / df.px.rolling(cruce[1]).mean() -1)*100

        pd.options.display.max_columns=15
        df['fw_20'] = (df.px.shift(-20) / df.px -1)*100

        # df = df.dropna().round(4)
        return df
    except:
        print('No se pudieron generar indicadores')
        return None

In [8]:
def traerData(symbol, days=5, interval='1min'):
    try:
        hasta = datetime.datetime.now()
        desde = hasta - datetime.timedelta(days=days)
        data = downloadTiingo(symbol, desde, hasta, interval=interval)
        return data
    except:
        print(f'No se pudo traer la data de {symbol} de los ultimos {days} dias')
        return None

In [9]:
def predecir(data, modelo):
    try:
        actual = generarIndicadores(data).iloc[-1,1:-1]
        y_pred = modelo.predict((actual,))[0]
        y_proba = modelo.predict_proba((actual,))[0]
        return y_pred, y_proba
    except:
        print('No se pudo predecir')
        return None, None

## Funciones de Ruteo

In [10]:
def pedirToken():
    url = 'https://api.invertironline.com/token'
    data = {"username" : iol_user, "password": iol_password, "grant_type":"password"}
    tk  = requests.post(url = url, data = data).json()
    return tk

In [11]:
def actualizarToken():
    global tk
    exp = datetime.datetime.strptime(tk['.expires'],'%a, %d %b %Y %H:%M:%S GMT')
    ahora = datetime.datetime.utcnow()
    tiempo = exp-ahora

    if tiempo.days != 0:
        print('Actualizando Token')
        tk = pedirToken()

    #print('Token actualizado')

In [12]:
def precioIOL(mercado, ticker):    
    actualizarToken() 
    url_base= 'https://api.invertironline.com/api/v2/'
    endpoint = mercado+'/Titulos/'+ticker+'/Cotizacion'
    url = url_base + endpoint
    headers = {"Authorization" : "Bearer "+ tk['access_token']}
    data = requests.get(url = url, headers = headers)
    
    try:
        data = data.json()
    except:
        print(f'No se pudo traer precio de {ticker} mercado {mercado}')
    return data

In [13]:
def comprar(ticker, q, precio, plazo ):
    actualizarToken()     
    vigencia = datetime.datetime.now() + datetime.timedelta(days=1)
    vigencia_str =datetime.datetime.strftime(vigencia, '%Y-%m-%d')
    params = {
      "mercado": "bCBA",
      "simbolo": ticker,
      "cantidad": q,
      "precio": precio,
      "plazo": plazo,
      "validez": vigencia_str
    }
    url_base= 'https://api.invertironline.com/api/v2/'
    endpoint = 'operar/comprar/'
    url = url_base + endpoint
    headers = {"Authorization" : "Bearer "+ tk['access_token']}    
    data = requests.post(url = url, headers = headers, json = params).json()
    return data

In [14]:
def vender(ticker, q, precio, plazo ):
    actualizarToken()     
    vigencia = datetime.datetime.now() + datetime.timedelta(days=1)
    vigencia_str = datetime.datetime.strftime(vigencia, '%Y-%m-%d')
    params = {
      "mercado": "bCBA",
      "simbolo": ticker,
      "cantidad": q,
      "precio": precio,
      "plazo": plazo,
      "validez": vigencia_str
    }
    url_base= 'https://api.invertironline.com/api/v2/'
    endpoint = 'operar/vender/'
    url = url_base + endpoint
    headers = {"Authorization" : "Bearer "+ tk['access_token']}    
    data = requests.post(url = url, headers = headers, json = params).json()
    return data

In [15]:
def compraAgresiva(symbol):
    data = precioIOL('bcba',symbol)
    try:
        punta_bid = data['puntas'][0]['precioCompra']
        punta_ask = data['puntas'][0]['precioVenta']
        id_compra = comprar(ticker=symbol, q=1, precio = punta_ask, plazo='t2')
        id_iol, px = id_compra['numeroOperacion'], punta_ask
    except:
        id_iol, px = None, None
        print('No se pudo enviar Orden de Compra')
        
    return id_iol, px

def ventaAgresiva(symbol):
    data = precioIOL('bcba',symbol)
    try:
        punta_bid = data['puntas'][0]['precioCompra']
        punta_ask = data['puntas'][0]['precioVenta']
        id_venta = vender(ticker=symbol, q=1, precio = punta_bid, plazo='t2')
        id_iol, px = id_venta['numeroOperacion'], punta_bid
    except:
        id_iol, px = None, None
        print('No se pudo enviar Orden de Venta')
        
    return id_iol, px

## Funciones estado sin BBDD (API)

In [16]:
def portafolio(pais):
    actualizarToken()     
    url_base= 'https://api.invertironline.com/api/v2/'
    endpoint = 'portafolio/'+pais
    url = url_base + endpoint
    headers = {"Authorization" : "Bearer "+ tk['access_token']}
    data = requests.get(url = url, headers = headers).json()    
    return(data['activos'])

In [17]:
def consultarTenencia(symbol):
    port = portafolio('Argentina')
    encontrados = sum([x['cantidad'] for x in port if x['titulo']['simbolo']==symbol])
    tenencia = True if encontrados > 0 else False
    return tenencia

## Funciones de Base de Datos

In [18]:
def conectarBBDD():
    sqlite_engine = create_engine('sqlite:///bot.db', echo=False)
    conn = sqlite_engine.connect()
    crear_tabla_trades = '''CREATE TABLE IF NOT EXISTS trades (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        ticker TEXT NOT NULL,
        px REAL NOT NULL,
        q INTEGER NOT NULL,
        hora_decimal REAL NOT NULL,
        tipo TEXT,
        id_iol TEXT
    );'''

    crear_tabla_predicciones = '''CREATE TABLE IF NOT EXISTS predicciones (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        ticker TEXT NOT NULL,
        proba_suba REAL NOT NULL,
        created_at datetime default current_timestamp
    );'''

    conn.execute(crear_tabla_trades)
    conn.execute(crear_tabla_predicciones)
    return conn

In [19]:
def addTrade(symbol, px, q, hora_decimal, tipo, id_iol):
    if id_iol:
        insertar = f'INSERT INTO trades VALUES(null, "{symbol}", {px}, {q}, {hora_decimal}, "{tipo}", "{id_iol}")'
    else:
        insertar = f'INSERT INTO trades VALUES(null, "{symbol}", {px}, {q}, {hora_decimal}, "{tipo}", null)'
        # Aca hay que hacer algo un poco mas serio
            #¿Pregunto al broker la tenencia?
            #¿Espero y vuelvo a preguntar?
            #¿Cierro todo?
        
    conn.execute(insertar)
    
def getLastTrade(symbol):
    ultimo_trade = f'SELECT * FROM trades WHERE ticker="{symbol}" ORDER BY id DESC LIMIT 1'
    trade = conn.execute(ultimo_trade).fetchone()
    return trade

In [20]:
def consultarTenenciaBBDD(symbol):
    trade = getLastTrade(symbol)
    tenencia = False
    if (trade):
        if trade[5] == 'Compra':
            tenencia = True
            
    return tenencia

In [21]:
def addPredict(symbol, proba_suba):
    insertar = f'INSERT INTO predicciones VALUES(null, "{symbol}", {proba_suba} , datetime("now","localtime"))'
    conn.execute(insertar)

# BOT

## Funcion principal

In [22]:
def monitorearDerrapes():
    vix = getVIX()
    
    salida = False
    if vix > 50:
        salida = True
    
    return salida


In [23]:
def ejecutar(modeloCompra):
    ahora = datetime.datetime.now()
    hora_decimal = round(ahora.hour + ahora.minute/60 + ahora.second/3600 + ahora.microsecond/(3.6*10**9) ,5)
    tenencia = consultarTenenciaBBDD('GGAL')
    
    if tenencia:
        # Si, estoy comprado
        hora_compra = getLastTrade('GGAL')[4]
        tiempo_tenencia = hora_decimal - hora_compra
        
        if tiempo_tenencia > 20/60:
            id_venta, px = ventaAgresiva('GGAL')
            addTrade('GGAL', px, 1, hora_decimal, 'Venta', id_venta)
            print(f'Vendido GGAL a {px} hora {hora_decimal}')
            time.sleep(60)            
        else:
            print(f'Esperando para vender a las {hora_compra+20/60} son las {hora_decimal}')
            time.sleep(60)
    
    else:
        # No estoy comprado
        data = traerData('GGAL')
        prediccion = predecir(data, modeloCompra)
        addPredict('GGAL', prediccion[1][1])
        if prediccion[0] == 1:
            id_compra, px = compraAgresiva('GGAL')
            addTrade('GGAL', px, 1, hora_decimal, 'Compra', id_compra)
            print(f'Comprado GGAL a {px} hora {hora_decimal}')
            time.sleep(60)
        else:
            print(f'Proba actual suba: {prediccion[1][1]:.2%} Esperando comprar, a las {hora_decimal}')
            time.sleep(60)     


## Pruebo Funciones por separado

In [33]:
modeloCompra = traerModelo('RF')
modeloCompra

RandomForestClassifier(criterion='entropy', max_depth=15)

In [34]:
data = traerData('GGAL')
data

,close,high,low,open
date,,,,
2021-07-12T13:30:00.000Z,8.230,8.230,8.230,8.230
2021-07-12T13:31:00.000Z,8.230,8.230,8.230,8.230
2021-07-12T13:32:00.000Z,8.230,8.230,8.230,8.230
2021-07-12T13:33:00.000Z,8.200,8.200,8.200,8.200
2021-07-12T13:34:00.000Z,8.200,8.200,8.200,8.200
...,...,...,...,...
2021-07-14T19:55:00.000Z,8.540,8.540,8.535,8.535
2021-07-14T19:56:00.000Z,8.550,8.555,8.540,8.545
2021-07-14T19:57:00.000Z,8.555,8.555,8.555,8.555


In [36]:
modeloCompra = traerModelo('RF')


(0, array([0.58013069, 0.41986931]))

In [41]:
%%timeit
data = traerData('GGAL')

1.01 s ± 7.82 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [43]:
%%timeit
prediccion = predecir(data, modeloCompra)

63.7 ms ± 4.31 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [37]:
conn = conectarBBDD()
consultarTenenciaBBDD('GGAL')

False

## Deamon: el demonio

In [38]:
HORA_INICIO_BOT = 23.55
HORA_FIN_BOT = 23.56

tk = pedirToken()
modeloCompra = traerModelo('RF')
conn = conectarBBDD()

while True:
    ahora = datetime.datetime.now()
    hora_decimal = round(ahora.hour + ahora.minute/60 + ahora.second/3600,5)
    
    if hora_decimal < HORA_INICIO_BOT:
        clear_output(wait=True)
        print(f'Esperando a las {HORA_INICIO_BOT}, Son las:',hora_decimal)
        time.sleep(1)
        # Aca puede ejecutar tareas de preparacion de rueda
        
    else:
        if hora_decimal < HORA_FIN_BOT:
            #if monitorearDerrapes() ==True:
            #    break
            ejecutar(modeloCompra)
            
        else:
            clear_output(wait=True)
            print(f'Son las {hora_decimal}, fin de horario de operaciones: {HORA_FIN_BOT}')

            # Aca puede ejecutar tareas de finalizacion del dia
            break


Esperando a las 23.55, Son las: 21.78583


KeyboardInterrupt: 

Potenciales mejoras al BOT-1

* Modelo predictivo de Compra:
    * Mas datos
    * Mejores indicadores
    * Metodo de Clusterizacion para encontrar acciones similares
    
* Modelo de decision:
    * Compra:
        * Mas de una prediccion
        * Decision en funcion de algo de mi cartera
    
    * Venta:
        * Por SL
        * Por TP
        * Timeout (*)
        * Modelos predictivos (pueden ser los mismos que en la compra)


* Monitores o dashboard del bot
* Screener de acciones como potenciales a tradear
* Asignacion de capital del Bot

* Monitoreo y breaks en caso de emergencias
